# Mimic II Dataset

https://physionet.org/content/mimiciii/1.4/

DOI (version 1.4):
https://doi.org/10.13026/C2HM2Q

DOI (latest version):
https://doi.org/10.13026/jbmn-w042

Topics:
critical care mimic electronic health records

Project Website:
https://mimic.physionet.org


## Data Description

MIMIC-III is a relational database consisting of 26 tables. Tables are linked by identifiers which usually have the suffix ‘ID’. For example, SUBJECT_ID refers to a unique patient, HADM_ID refers to a unique admission to the hospital, and ICUSTAY_ID refers to a unique admission to an intensive care unit.

Charted events such as notes, laboratory tests, and fluid balance are stored in a series of ‘events’ tables. For example the OUTPUTEVENTS table contains all measurements related to output for a given patient, while the LABEVENTS table contains laboratory test results for a patient.

Tables prefixed with ‘D_’ are dictionary tables and provide definitions for identifiers. For example, every row of CHARTEVENTS is associated with a single ITEMID which represents the concept measured, but it does not contain the actual name of the measurement. By joining CHARTEVENTS and D_ITEMS on ITEMID, it is possible to identify the concept represented by a given ITEMID.

Developing the MIMIC data model involved balancing simplicity of interpretation against closeness to ground truth. As such, the model is a reflection of underlying data sources, modified over iterations of the MIMIC database in response to user feedback. Care has been taken to avoid making assumptions about the underlying data when carrying out transformations, so MIMIC-III closely represents the raw hospital data.

Broadly speaking, five tables are used to define and track patient stays: ADMISSIONS; PATIENTS; ICUSTAYS; SERVICES; and TRANSFERS. Another five tables are dictionaries for cross-referencing codes against their respective definitions: D_CPT; D_ICD_DIAGNOSES; D_ICD_PROCEDURES; D_ITEMS; and D_LABITEMS. The remaining tables contain data associated with patient care, such as physiological measurements, caregiver observations, and billing information.

In some cases it would be possible to merge tables—for example, the D_ICD_PROCEDURES and CPTEVENTS tables both contain detail relating to procedures and could be combined—but our approach is to keep the tables independent for clarity, since the data sources are significantly different. Rather than combining the tables within MIMIC data model, we suggest researchers develop database views and transforms as appropriate.

## Usage Notes

MIMIC-III is provided as a collection of comma separated value (CSV) files, along with scripts to help with importing the data into database systems including PostreSQL, MySQL, and MonetDB. As the database contains detailed information regarding the clinical care of patients, it must be treated with appropriate care and respect. Researchers are required to formally request access via a process documented on the MIMIC website. There are two key steps that must be completed before access is granted:

- the researcher must complete a recognized course in protecting human research participants that includes Health Insurance Portability and Accountability Act (HIPAA) requirements.
- the researcher must sign a data use agreement, which outlines appropriate data usage and security standards, and forbids efforts to identify individual patients.

Approval requires at least a week. Once an application has been approved the researcher will receive emails containing instructions for downloading the database from PhysioNetWorks, a restricted access component of PhysioNet.

In [50]:
%pip install duckdb

Note: you may need to restart the kernel to use updated packages.


In [51]:
import duckdb

## Import MIMIC-III Data Into DuckDB

In [52]:
if False:
    conn = duckdb.connect('mimiciii.db', read_only=False)
    try:
        df = conn.execute("""
            -- To get unique patients
            CREATE TABLE IF NOT EXISTS PATIENTS AS
                FROM read_csv_auto(
                            'datasets/physionet.org/files/mimiciii/1.4/PATIENTS.csv.gz',
                            header=True,
                            union_by_name=true,
                            files_to_sniff=-1,
                            filename=true
                            );

            -- To get a single visit and dianosis. Without this it's hard
            -- to limit a patient diagnosis to an event
            CREATE TABLE IF NOT EXISTS ADMISSIONS AS
                FROM read_csv_auto(
                            'datasets/physionet.org/files/mimiciii/1.4/ADMISSIONS.csv.gz',
                            header=True,
                            union_by_name=true,
                            files_to_sniff=-1,
                            filename=true
                            );

            -- Not 100% I need this one just yet
            CREATE TABLE IF NOT EXISTS CHARTEVENTS AS
                FROM read_csv_auto(
                            'datasets/physionet.org/files/mimiciii/1.4/CHARTEVENTS.csv.gz',
                            header=True,
                            union_by_name=true,
                            files_to_sniff=-1,
                            filename=true,
                            sample_size=-1
                            );

            -- ICD9_CODE (used to get diagnosis id)
            CREATE TABLE IF NOT EXISTS DIAGNOSES_ICD AS
                FROM read_csv_auto(
                            'datasets/physionet.org/files/mimiciii/1.4/DIAGNOSES_ICD.csv.gz',
                            header=True,
                            union_by_name=true,
                            files_to_sniff=-1,
                            filename=true,
                            sample_size=-1
                            );

            -- ICD9_CODE (diagnosis text)
            CREATE TABLE IF NOT EXISTS D_ICD_DIAGNOSES AS
                FROM read_csv_auto(
                            'datasets/physionet.org/files/mimiciii/1.4/D_ICD_DIAGNOSES.csv.gz',
                            header=True,
                            union_by_name=true,
                            files_to_sniff=-1,
                            filename=true,
                            sample_size=-1
                            );

            -- to get the patients chart.
            CREATE TABLE IF NOT EXISTS NOTEEVENTS AS
                FROM read_csv_auto(
                            'datasets/physionet.org/files/mimiciii/1.4/NOTEEVENTS.csv.gz',
                            header=True,
                            union_by_name=true,
                            files_to_sniff=-1,
                            filename=true,
                            sample_size=-1
                            );
        """).df()
        print(df)

        # conn.execute("COPY mytable TO 'output.csv' (HEADER, DELIMITER ',')")

    except Exception as ex:
        print(ex)
    finally:
        conn.close()


## Classes in the Vector Database as Integers

In [53]:
classes = [
    "Intellectual Developmental Disorders",                        # 0
    "Communication Disorders",
    "Autism Spectrum Disorder",
    "Attention-Deficit/Hyperactivity Disorder",
    "Specific Learning Disorder",
    "Motor Disorders",
    "Other Neurodevelo mental Disorders p",
    "Schizophrenia Spectrum and Other Psychotic Disorders",
    "Catatonia",
    "Bipolar and Related Disorders",
    "Depressive Disorders",                                        # 10
    "Anxiety Disorders",
    "Obsessive-Compulsive and Related Disorders",
    "Trauma- and Stressor-Related Disorders",
    "Dissociative Disorders",
    "Somatic Symptom and Related Disorders",
    "Feeding and Eating Disorders",
    "Elimination Disorders",
    "Sleep-Wake Disorders",
    "Breathing-Related Sleep Disorders",
    "Parasomnias",                                                # 20
    "Sexual Dysfunctions",
    "Gender Dysphoria",
    "Disruptive, Impulse-Control, and Conduct Disorders",
    "Neurocognitive Disorders",
    "Personality Disorders",
    "Cluster A Personality Disorders",
    "Cluster B Personality Disorders",
    "Cluster C Personalit Disorders y",
    "Other Personality Disorders",
    "Paraphilic Disorders",                                       # 30
    "Other Mental Disorders and Additional Codes",
    "Additional Codes",
    "Medication-Induced Movement Disorders and Other Adverse Effects of Medication",
    "Other Conditions That May Be a Focus of Clinical Attention", # 34
]
print(classes[0])
print(classes[10])
print(classes[20])
print(classes[30])
print(classes[34])

Intellectual Developmental Disorders
Depressive Disorders
Parasomnias
Paraphilic Disorders
Other Conditions That May Be a Focus of Clinical Attention


## Create Patient Record Data Linked to Diagnosis

In [54]:
# "Depressive Disorders" == "Depressive disorder NEC"
#    "Anxiety Disorders" == "Anxiety state NOS"
conn = duckdb.connect('mimiciii.db', read_only=True)
result = conn.sql("""
    SELECT
        CASE
            WHEN dd.short_title = 'Depressive disorder NEC' THEN 10
            WHEN dd.short_title = 'Anxiety state NOS' THEN 11
        END AS class_number,
        count(*)
    FROM patients p
    JOIN diagnoses_icd d ON p.subject_id = d.subject_id
    JOIN d_icd_diagnoses dd ON d.icd9_code = dd.icd9_code
    JOIN (
        SELECT t2.subject_id,
            ROW_NUMBER() OVER(PARTITION BY t2.subject_id) as rn
        FROM noteevents t2
    ) AS t2 ON (p.subject_id = t2.subject_id AND t2.rn = 1)
    WHERE (short_title = 'Depressive disorder NEC'
       OR short_title = 'Anxiety state NOS')
    GROUP BY class_number
""").fetchall()
# Depressive  3412
#    Anxiety  1574
# -----------------
#             4986
print(result)
conn.close()


[(10, 3412), (11, 1574)]


In [57]:
# "Depressive Disorders" == "Depressive disorder NEC"
#    "Anxiety Disorders" == "Anxiety state NOS"
conn = duckdb.connect('mimiciii.db', read_only=True)
conn.sql("""
    COPY (
        SELECT
            -- p.subject_id,
            GROUP_CONCAT(ne.text) AS chart_text,
            CASE
                WHEN dd.short_title = 'Depressive disorder NEC' THEN 10
                WHEN dd.short_title = 'Anxiety state NOS' THEN 11
            END AS class_number
        FROM patients p
        JOIN diagnoses_icd d ON p.subject_id = d.subject_id
        JOIN d_icd_diagnoses dd ON d.icd9_code = dd.icd9_code
        JOIN noteevents ne ON p.subject_id = ne.subject_id
        WHERE (short_title = 'Depressive disorder NEC'
           OR short_title = 'Anxiety state NOS')
        AND (ne.category = 'Nursing'
           OR ne.category = 'Nursing/other'
           OR ne.category = 'General'
           OR ne.category = 'Social Work'
           OR ne.category = 'Pharmacy'
           OR ne.category = 'Consult'
           OR ne.category = 'Case Management'
           OR ne.category = 'Discharge summary'
           OR ne.category = 'Physician'
         )
        GROUP BY p.subject_id, class_number
    ) TO 'datasets/text_classes.csv' (FORMAT csv, DELIMITER '|', HEADER, QUOTE '"');
""") #.write_csv('datasets/text_classes.csv', sep="|", escapechar="~")
# display(len(result)) # ==> 4353
conn.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))